In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.auto import tqdm

In [47]:
import requests

url = "https://studenttheses.universiteitleiden.nl/search"

headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en,en-GB;q=0.9",
    "Cache-Control": "no-cache",
    "Connection": "keep-alive",
    "Cookie": "SSESSc128199592f79081a0f8de92b9e4cacf=Ede1Q4tCjfh4IHTiaDzBhpWJEns_oyvz1tBVZAirPHA",
    "Dnt": "1",
    "Host": "studenttheses.universiteitleiden.nl",
    "Pragma": "no-cache",
    "Sec-Ch-Ua": '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}




In [48]:
params = {
    "islandora_solr_search_navigation": "1",
    "display": "grid",
    "f[0]": r'mods_name_personal_affiliation_faculty_ms:"Faculty\ of\ Social\ &\ Behavioural\ Sciences"',
    
}

response = requests.get(url, headers=headers, params=params)
soup = BeautifulSoup(response.content, "html.parser")

#pages
pages = soup.find("div", {'id': 'islandora-solr-result-count'})

#loop pages
num_pages = int(pages.text.split(" ")[-1].replace("(", "").replace(")", "").replace(",", ""))
#24 per page
num_pages = num_pages//24 + 1

#loop pages
FMG_ids = []
params = {
    "islandora_solr_search_navigation": "1",
    "display": "grid",
    "f[0]": r'mods_name_personal_affiliation_faculty_ms:"Faculty\ of\ Social\ &\ Behavioural\ Sciences"',
    "page": 0
}

for page in tqdm(range(0, num_pages)):
    params['page'] = page
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.content, "html.parser")
    for link in soup.find_all("dd", {"class": "solr-grid-caption"}):
        link = link.find("a")
        FMG_ids.append(link['href'])

print(len(FMG_ids))

  0%|          | 0/851 [00:00<?, ?it/s]

20406


In [49]:


params = {
    "islandora_solr_search_navigation": "1",
    "display": "grid",
    "f[0]": r'mods_name_personal_affiliation_faculty_ms:"Faculty\ of\ Humanities"',
    "page": 0
}
response = requests.get(url, headers=headers, params=params)
soup = BeautifulSoup(response.content, "html.parser")

#pages
pages = soup.find("div", {'id': 'islandora-solr-result-count'})
print(pages.text)
num_pages = int(pages.text.split(" ")[-1].replace("(", "").replace(")", "").replace(",", ""))
#24 per page
num_pages = num_pages//24 + 1

H_ids = []

#loop pages
for page in tqdm(range(0, num_pages)):
    params['page'] = page
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.content, "html.parser")
    for link in soup.find_all("dd", {"class": "solr-grid-caption"}):
        link = link.find("a")

        H_ids.append(link['href'])

(1 - 24 of 17,964)


  0%|          | 0/749 [00:00<?, ?it/s]

In [55]:
#save all ids to a txt
with open("ids.txt", "w") as f:
    for id in FMG_ids:
        f.write(id + "\n")
    for id in H_ids:
        f.write(id + "\n")
        

In [50]:
#dataframe title,author,supervisors,journal,date,faculty,program,department,language,abstract,keywords

df_FMG = pd.DataFrame(columns=['title','author','supervisors','date','faculty','program','type','language','abstract','keywords'])
for i in tqdm(FMG_ids):
    url = "https://studenttheses.universiteitleiden.nl" + i
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    try:
        title = soup.find("dd", {"class": "title"}).text.strip()
    except:
        title = None
    
    try:
        container = soup.find("dd", {"class": "first author"})
        author = container.find("span", {"class": "name"}).text.strip()
        year = container.find("span", {"class": "year"}).text.strip().replace("(", "").replace(")", "")
    except:
        author = None
        year = None

    try:
        container = soup.find("dl", {"title": "Supervisors"})
        supervisors = container.find("dd")
        supervisors = supervisors.findAll('a')
        supervisors = [i.text.strip() for i in supervisors]
    except:
        supervisors = None

    try:
        container = soup.find("dl", {"title": "Faculty"})
        faculty = container.find("dd").text.strip()
    except:
        faculty = None

    try:
        container = soup.find("dd", class_='genre')
        a = container.findAll('a')
        type = a[0].text.strip()
        program = a[1].text.strip()
    except:
        type = None
        program = None

    try:
        container = soup.find("dl", {"title": "Language"})
        language = container.find("dd").text.strip()
    except:
        language = None

    try:
        container = soup.find("dd", {"class": "abstract"})
        abstract = container.find("span")
        abstract = abstract.findAll('span')[1]
        abstract = abstract.text.strip()
        #remove Show less
        abstract = abstract.split("Show less")[0]
    except:
        abstract = None

    try:
        container = soup.find("dd", {"class": "topics"})
        keywords = container.findAll("div", class_='tag')

        keywords = [i.text.strip() for i in keywords]
    except:
        keywords = None

    data = {
        'title': title,
        'author': author,
        'supervisors': supervisors,
        'date': year,
        'faculty': faculty,
        'program': program,
        'type': type,
        'language': language,
        'abstract': abstract,
        'keywords': keywords
    }
    df_FMG = df_FMG._append(data, ignore_index=True)
    

  0%|          | 0/20406 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df_H = pd.DataFrame(columns=['title','author','supervisors','date','faculty','program','type','language','abstract','keywords'])
for i in tqdm(H_ids):
    url = "https://studenttheses.universiteitleiden.nl" + i
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    try:
        title = soup.find("dd", {"class": "title"}).text.strip()
    except:
        title = None
    
    try:
        container = soup.find("dd", {"class": "first author"})
        author = container.find("span", {"class": "name"}).text.strip()
        year = container.find("span", {"class": "year"}).text.strip().replace("(", "").replace(")", "")
    except:
        author = None
        year = None

    try:
        container = soup.find("dl", {"title": "Supervisors"})
        supervisors = container.find("dd")
        supervisors = supervisors.findAll('a')
        supervisors = [i.text.strip() for i in supervisors]
    except:
        supervisors = None

    try:
        container = soup.find("dl", {"title": "Faculty"})
        faculty = container.find("dd").text.strip()
    except:
        faculty = None

    try:
        container = soup.find("dd", class_='genre')
        a = container.findAll('a')
        type = a[0].text.strip()
        program = a[1].text.strip()
    except:
        type = None
        program = None

    try:
        container = soup.find("dl", {"title": "Language"})
        language = container.find("dd").text.strip()
    except:
        language = None

    try:
        container = soup.find("dd", {"class": "abstract"})
        abstract = container.find("span")
        abstract = abstract.findAll('span')[1]
        abstract = abstract.text.strip()
        #remove Show less
        abstract = abstract.split("Show less")[0]
    except:
        abstract = None

    try:
        container = soup.find("dd", {"class": "topics"})
        keywords = container.findAll("div", class_='tag')

        keywords = [i.text.strip() for i in keywords]
    except:
        keywords = None

    data = {
        'title': title,
        'author': author,
        'supervisors': supervisors,
        'date': year,
        'faculty': faculty,
        'program': program,
        'type': type,
        'language': language,
        'abstract': abstract,
        'keywords': keywords
    }
    df_H = df_H._append(data, ignore_index=True)

    

In [ ]:
#concat and csv
df = pd.concat([df_FMG, df_H])
df.to_csv("theses.csv", index=False)
